In [56]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import utils
from nets.NetMLPLowRank import NetMLPLowRank

In [62]:
batch_size = 128
batch_size_test = 1000

train_loader, test_loader = utils.build_dataset(dataset='MNIST')

In [63]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
lr = 0.01
n_epochs = 100

model = NetMLPLowRank(K=[2], d=[2])
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

for epoch in range(n_epochs):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.332648
Train Epoch: 0 [20000/60000 (33%)]	Loss: 1.394118
Train Epoch: 0 [40000/60000 (67%)]	Loss: 1.245764

Test set: Average loss: -3.2692, Accuracy: 6179/10000 (62%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.981461
